# TP Algo Noté Mots 💻
*Benoit Kezel / Allan Des Courtils*



---



## Consignes :
```
Considérez le problème suivant : 

Soit M un mot du dictionnaire (chaine de caractères). 
On définit L(M) comme le nombre de caractères différents de M. ex : D("Polytech") = 8, D("kayak") = 3.7

Trouvez 5 mots de longueur 5 dont la valeur D de leur concaténation = 25. (5 mots ayant que des caractères distincts).
```

# Solution finale : Les univers

On utilise des "univers" qui nous permettent de créer une multitude de possibilités. 
Chaque univers est composé de sa propre liste de mot avec ses propres lettres déjà utilisées.
Lorsqu'on traite un mot, on regarde si on peut l'insérer dans un ou plusieurs univers. Si on ne peut l'insérer nul part, on créer un nouvel univers avec ce mot en premier puis on backtrack.

Lorsque une combinaison de 4 mots est trouvée au sein d'un univers, on recherche un 5ème mot manquant à partir des lettres disponibles

`⏰ Temps d'éxecution moyen : 50 millisecondes`

In [1]:
import urllib.request as urllib2
import ssl
import time

# Protocole pour récupérer la liste des mots stockée en ligne
ssl._create_default_https_context = ssl._create_unverified_context

words = []

# On récupère la liste des mots triés par rareté (voir bloc Annexe ci-dessous)
for line in urllib2.urlopen("https://nextcloud.kyserver.fr/s/jSAx23JEz97S25b/download/words5lettersSorted.txt"):
    words.append(str(line.decode("utf-8").strip()))

start =  time.time()

lettresXmots = dict()

# On crée un dictionnaire lettre/mot
for word in words:
  for letter in word:
    if letter not in lettresXmots.keys(): lettresXmots[letter] = [word]
    else:  lettresXmots[letter].append(word)

universes = []

lettersUsed = ""
wordsUsed = []

# On créer un premier univers par défaut
universes.append([lettersUsed, wordsUsed])
wordsUniverseCreated = []
motDeQuatres = dict()

findFive = False
#Tant que l'on ne trouve pas 5 mots, on reitere
while not findFive:
  i = 0
  while True:
      inserted = False
      find = False
      for universe in universes:
          wordOk = True
          # Boucle pour vérifier si le mot peut être insérer dans l'univers courant
          for letter in words[i]:
              if letter in universe[0]:
                  wordOk = False
                  break
          # Si on peut insérer le mot
          if(wordOk):
              inserted = True
              universe[0] += words[i]
              universe[1].append(words[i])
          # On vérifie si l'univers courant possède désormais le nombre de mots qui nous intéresse (4 dans notre cas)
          if len(universe[1]) == 4 and universe[1] not in motDeQuatres.values():
              find = True
              break

      # Si le mot n'a pas été inséré dans aucun univers
      if not inserted:
          # Si on n'a pas déjà créer un univers pour ce mot
          if words[i] not in wordsUniverseCreated:
              # On créer un nouvel univers avec ce mot dedans en premier
              lettersUsed = words[i]
              wordsUsed = [words[i]]
              universes.append([lettersUsed, wordsUsed])
              wordsUniverseCreated.append(words[i])
              # Backtracking, on reparcours les mots depuis le début
              i = -1
              
      # On sauvegarde la première liste de 4 mots
      if find:
          motDeQuatres[universe[0]] = universe[1]
          currentList = universe[0]
          break
      i += 1


  motDeCinq = motDeQuatres[currentList]
  lettreUtilise = currentList
  # on sauvegarde les lettres manquantes
  lettreManquante = list(set("abcdefghijklmnopqrst") - set(lettreUtilise))
  # on boucle dessus
  for lettre in lettreManquante:
    # on parcours les mots contenant la lettre manquante courante
    for word in lettresXmots[lettre]:
      wordOk = True
      for lettre in word:
        if lettre in lettreUtilise:
          wordOk = False
          break
      
      if wordOk:
        lettreUtilise += word
        motDeCinq.append(word)
        findFive = True
        break
    if findFive: break

end = time.time()
print(f"Lettres utilisées {sorted(lettreUtilise)}")
print(f"Mots utilisés {motDeCinq}")
print("Temps d'éxecution : ", round((end-start)*1000,2), "ms")

Lettres utilisées ['a', 'b', 'c', 'd', 'e', 'f', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Mots utilisés ['nymph', 'fldxt', 'vejoz', 'quick', 'braws']
Temps d'éxecution :  31.04 ms


---
# Annexe : Algo de tri des mots par rareté

Pour optimiser la solution, nous avons trié les mots par rareté au niveau des lettres qui le compose. 

Le fichier original n'étant pas dans le collab, il n'est pas possible de l'éxectuer cependant nous l'avons mis pour pouvoir regarder la logique

In [2]:
with open('words5letters.txt', 'r') as f:
    for line in f:
        words.append(line.strip())

frequenceLettre = dict()

for word in words:
  for letter in word:
    if letter not in frequenceLettre.keys(): frequenceLettre[letter] = 1
    else: frequenceLettre[letter] += 1

def calculateValue(word): return sum([frequenceLettre[l] for l in word])

valueWord = list()
for word in words:
  valueWord.append([word, calculateValue(word)])

valueWord.sort(key=lambda x: x[1])
words = [v[0] for v in valueWord]

FileNotFoundError: [Errno 2] No such file or directory: 'words5letters.txt'